In [238]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd

In [249]:
def get_soup_content(url):
    exe_path = {"executable_path": "./bin/chromedriver"}

    with Browser("chrome", **exe_path, headless=True) as browser:
        browser.visit(url)
        content = browser.html
        return BeautifulSoup(content, 'html.parser')

    
def memoized(value, storage):
    if value not in storage:
        storage[value] = value
        return storage[value]
    return ''


In [250]:
# Scrape the content for Mars news:
news_content = get_soup_content("https://mars.nasa.gov/news/")


In [251]:
# Data Wrangling for Mars news: 

content = news_content.find_all(class_="list_text")

# Get final response:
news_title = [element.find(class_="content_title").get_text() for element in content if element.find(class_="content_title")]
news_p = [element.find(class_="article_teaser_body").get_text() for element in content if element.find(class_="article_teaser_body")]


In [226]:
# Scrape the content for Mars feature image:
jpl_content = get_soup_content("https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars")


In [227]:
# Data Wrangling for Mars feature image:
featured_image_url = jpl_content.find(class_="carousel_item")["style"].split(":")[1].split("'")[1]

# Final response:
featured_image_url = "https://www.jpl.nasa.gov{}".format(jpl_style)


In [228]:
# Scrape the content for Mars tweets:
tweet_content = get_soup_content("https://twitter.com/marswxreport?lang=en")


In [229]:
# Data Wrangling for Mars tweets:
mars_weather = tweet_content.find(class_="TweetTextSize").get_text()


In [242]:
# Scrape the content for Mars facts:
facts_content = get_soup_content("http://space-facts.com/mars/")


In [248]:
# Data Wrangling for Mars facts:

facts_table = facts_content.find(id="tablepress-mars")

facts_columns = [title.get_text() for title in facts_table.find_all("td", attrs={"class": "column-1"})]
facts = [description.get_text() for description in facts_table.find_all("td", attrs={"class": "column-2"})]

# Final response:
facts_pd = pd.DataFrame([facts], columns =facts_columns)


In [177]:
# Scrape the content for Mars hemisphere data:
hemis_content = get_soup_content("https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars")


In [194]:
# Data Wrangling for Mars hemisphere data:

base_url = "https://astrogeology.usgs.gov{}"
html_hemis = hemis_content.find_all(class_="itemLink")

storage = {}
url_list = [base_url.format(link["href"]) for link in html_hemis if memoized(link["href"], storage)]

wide_image_list = []
for url in url_list:
    hemis_image_wide = get_soup_content(url)
    wide_image_list.append(base_url.format(hemis_image_wide.find(class_="wide-image")["src"]))

title_list = [title.get_text() for title in hemis_content.find_all("h3")]

# Final response:
hemis_dic = [{"title": key, "img_url": value} for (key, value) in list(zip(title_list, wide_image_list))]
